In [1]:
import urllib.request
from bs4 import BeautifulSoup
import csv
from time import sleep
import pandas as pd
import json
import urllib.request
import os
from PIL import Image
import glob

import yaml

prefix = "https://nakamura196.github.io/iiif0"

dir = "/Users/nakamura/git/hi/iiif0"

# dir = "../docs/files/original"

tile = "512"

f = open('tmp/convert.sh', 'w')
writer = csv.writer(f, lineterminator='\n')


files = glob.glob(dir+'/docs/files/original/**/*', recursive=True)


for i in range(len(files)):

    file = files[i]

    if not os.path.isfile(file):
        continue

    opath = file.replace("/original/", "/tile/")
    tmp = os.path.split(opath)
    odir = tmp[0]

    root, ext = os.path.splitext(opath)

    info_path = opath.replace(ext, "/info.json")

    if not os.path.exists(info_path):
        print(info_path)

        # os.makedirs(odir, exist_ok=True)

        line = "echo "+str(i+1)+"/"+str(len(files))
        writer.writerow([line])

        line = "mkdir -p "+odir
        writer.writerow([line])

        p = odir.replace(dir+"/docs", prefix)

        line = "python iiif_static/iiif_static.py  -d "+odir+" -t "+str(tile)+"  -p "+p+" "+file
        writer.writerow([line])
    

f.close()



/Users/nakamura/git/hi/iiif0/docs/files/tile/kouwazukan/kouwazukan/info.json
/Users/nakamura/git/hi/iiif0/docs/files/tile/kouwazukan/kouwazukan/info.json


In [2]:
!sh tmp/convert.sh

2/3
iiif_static.py: source file: /Users/nakamura/git/hi/iiif0/docs/files/original/kouwazukan/kouwazukan.jpeg
iiif.static: /Users/nakamura/git/hi/iiif0/docs/files/tile/kouwazukan / kouwazukan/0,0,512,512/512,/0/default.jpg
iiif.static: /Users/nakamura/git/hi/iiif0/docs/files/tile/kouwazukan / kouwazukan/0,512,512,512/512,/0/default.jpg
iiif.static: /Users/nakamura/git/hi/iiif0/docs/files/tile/kouwazukan / kouwazukan/0,1024,512,512/512,/0/default.jpg
iiif.static: /Users/nakamura/git/hi/iiif0/docs/files/tile/kouwazukan / kouwazukan/0,1536,512,512/512,/0/default.jpg
iiif.static: /Users/nakamura/git/hi/iiif0/docs/files/tile/kouwazukan / kouwazukan/0,2048,512,512/512,/0/default.jpg
iiif.static: /Users/nakamura/git/hi/iiif0/docs/files/tile/kouwazukan / kouwazukan/0,2560,512,512/512,/0/default.jpg
iiif.static: /Users/nakamura/git/hi/iiif0/docs/files/tile/kouwazukan / kouwazukan/0,3072,512,512/512,/0/default.jpg
iiif.static: /Users/nakamura/git/hi/iiif0/docs/files/tile/kouwazukan / kouwazukan/0

In [4]:
import urllib.request
from bs4 import BeautifulSoup
import csv
from time import sleep
import pandas as pd
import json
import urllib.request
import os
from PIL import Image
import glob

import yaml

dir = ""
id = "kouwazukan"
label = "抗倭図巻"
vhint = "right-to-left"

files = glob.glob("../docs/files/tile/{}{}/**/info.json".format(dir, id), recursive=True)

files = sorted(files)

canvases = []

for i in range(len(files)):
    file = files[i]

    index = i + 1

    print(file)

    with open(file) as f:
        df = json.load(f)

        th_width = df["sizes"][0]["width"]
        th_height = df["sizes"][0]["height"]

        image_id = df["@id"]

        canvas = {
          "label": "[{}]".format(index),
          "width": df["width"],
          "height": df["height"],
          "@type": "sc:Canvas",
          "@id": "https://nakamura196.github.io/iiif0/iiif/{}/canvas/p{}".format(id, index),
          "images": [
            {
              "@type": "oa:Annotation",
              "on": "https://nakamura196.github.io/iiif0/iiif/{}/canvas/p{}".format(id, index),
              "motivation": "sc:painting",
              "resource": {
                "@type": "dctypes:Image",
                "format": "image/jpeg",
                "width": df["width"],
                "height": df["height"],
                "@id": image_id + "/full/full/0/default.jpg",
                "service": {
                  "@context": "http://iiif.io/api/image/2/context.json",
                  "@id": image_id,
                  "profile": "http://iiif.io/api/image/2/level0.json"
                }
              }
            }
          ],
          "thumbnail": {
            "@id": image_id + "/full/{},/0/default.jpg".format(th_width),
            "@type": "dctypes:Image",
            "format": "image/jpeg",
            "width": th_width,
            "height": th_height
          }
        }

        canvases.append(canvas)

manifest = {
  "label": label,
  "@id": "https://nakamura196.github.io/iiif0/iiif/{}.json".format(id),
  "@type": "sc:Manifest",
  "@context": "http://iiif.io/api/presentation/2/context.json",
  "metadata": [],
  "within": "",
  "logo": "",
  "description": "",
  "viewingHint": "",
  "viewingDirection": vhint,
  "license": "http://creativecommons.org/licenses/by/4.0/",
  "sequences": [
    {
      "@type": "sc:Sequence",
      "label": "Current Page Order",
      "@id": "https://nakamura196.github.io/iiif0/iiif/{}/sequence/normal".format(id),
      "canvases": canvases
    }
  ]
}

path = "../docs/iiif/{}{}.json".format(dir, id)

os.makedirs(os.path.dirname(path), exist_ok=True)

fw = open(path, 'w')
json.dump(manifest, fw, ensure_ascii=False, indent=4, sort_keys=True, separators=(',', ': '))

../docs/files/tile/kouwazukan/info.json
